In [1]:
import Geometry as gm

SCALE = 50
DEVIATION = 10

WHITE = (255, 255, 255)
RED = (255, 0, 0)
GREEN = (0, 255, 0)
BLUE = (0, 0, 255)
PURPLE = (255, 0, 255)
CYAN = (0, 255, 255)
YELLOW = (255, 255, 0)
BLACK = (0, 0, 0)

def def_value():
    return None

layers = []
n = 5
for i in range(n):
    layers.append(gm.DCEL())
    #Temporal dictionaries to store the half-edges while building the layer
    edges = dict()
    vertices = dict()
    faces = dict()
    #Reads input file
    file_name = f"inputs/layer0{i + 1}"
    
    #Ignores overhead of the files
    ari = open(file_name + ".aristas").readlines()[4:]
    vert = open(file_name + ".vertices").readlines()[4:]
    car = open(file_name + ".caras").readlines()[4:]
    
    #Start building the layer from the vertices
    for line in vert:
        name, x, y, incident = line.split()
        vertices[name] = gm.Vertex(float(x) * SCALE + DEVIATION, float(y)*SCALE + DEVIATION)
        #We don't have half-edges so we don't add incident edge yet
    #Start making the half-edges
    for line in ari:
        name, ori, twin, face, next, prev = line.split()
        #Create based on origin point
        edges[name] = gm.Half_Edge(vertices[ori], layer=i+1)
    #Add incident edges
    for line in vert:
        name, x, y, incident = line.split()
        vertices[name].incident = incident
        layers[i].add_vertex(vertices[name])
    #Create the faces
    for line in car:
        name, internal, external = line.split()
        internal = internal.replace("[", "").replace("]", "")
        external = external.replace("[", "").replace("]", "")
        icomp = []
        if len(internal.split(",")) >= 2:
            internal = internal.split(",")
            for comp in internal:
                icomp.append(edges[comp])
        if internal == "None": internal = None
        if external == "None": external = None
        # Adds pointers to different cycles
        if icomp:
            faces[name] = gm.Face(external, icomp)
        else:
            faces[name] = gm.Face(external, internal)
        layers[i].add_face(faces[name])
    #Complete information about the half-edges
    for line in ari:
        name, ori, twin, face, next, prev = line.split()
        #Create based on origin point
        edges[name].twin = edges[twin]
        edges[name].next = edges[next]
        edges[name].prev = edges[prev]
        edges[name].face = faces[face]
        layers[i].add_half_edge(edges[name])
    
#for layer in layers:
#    print(layer)
    
intersections = gm.calcIntersections(layers)

area = gm.DCEL().overlay(layers, intersections)
    
print(area)

List of edges [Edge from (160.0, 260.0) to (159.99999999998312, 291.2499999999894), Edge from (160.0, 460.0) to (159.99999999999, 360.00000000001), Edge from (160.0, 460.0) to (260.0, 410.0), Edge from (260.0, 410.0) to (160.0, 460.0), Edge from (260.0, 410.0) to (260.0000000000056, 353.75000000000347), Edge from (260.0, 260.0) to (260.000000000015, 259.999999999985), Edge from (260.0, 260.0) to (260.0, 260.0), Edge from (160.0, 260.0) to (260.0, 260.0), Edge from (410.0, 210.0) to (510.000000000035, 159.9999999999825), Edge from (510.0, 160.0) to (510.000000000035, 159.9999999999825), Edge from (510.0, 160.0) to (510.000000000035, 160.0000000000175), Edge from (410.0, 110.0) to (410.0, 110.0), Edge from (410.0, 110.0) to (409.99999999999, 110.00000000001), Edge from (410.0, 210.0) to (409.99999999999, 110.00000000001), Edge from (410.0, 460.0) to (460.0, 560.0), Edge from (460.0, 560.0) to (410.0, 460.0), Edge from (460.0, 560.0) to (660.0, 535.0), Edge from (660.0, 535.0) to (460.0, 

In [2]:
import pygame

# Helper function to draw in pygame screen
def draw_point(screen, vertex: gm.Vertex, color):
    pygame.draw.circle(screen, color, vertex.coord, 2)

def draw_edge(screen, edge: gm.Half_Edge, color):
    pygame.draw.line(screen, color, edge.origin.coord, edge.twin.origin.coord)
    
def draw_layer(screen, layer: gm.DCEL, color):
    edges = layer.half_edges
    for edge in edges:
        draw_edge(screen, edge, color)
    for intersection in intersections:
        pygame.draw.circle(screen, BLACK, (intersection.x,intersection.y), 2)

pygame 2.3.0 (SDL 2.24.2, Python 3.10.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
pygame.init()
screen_width = 800
screen_height = 600
screen = pygame.display.set_mode((screen_width, screen_height))
screen.fill(WHITE)

draw_layer(screen, area, BLUE)

pygame.display.update()

while True:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            quit()


error: video system not initialized

: 